# DQN Approach

## 0. Import the necessary packages

In [1]:
import math
import random
import matplotlib
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if GPU is to be used
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# if metal is to be used
# device = torch.device("metal" if torch.cuda.is_available() else "cpu")

if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [ ]:
from actor_critic_simulation.real_environment import Environment

## 1. Creating the architecture of the Q-Network and Agent

In [8]:
class DQN(nn.Module):

    def __init__(self, state_size, action_size, seed, fc1_units=128, fc2_units=128):
        super(DQN, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

In [9]:
class ReplayBuffer:
    def __init__(self, action_size, buffer_size, batch_size, seed):
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done):
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
  
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        return len(self.memory)

In [64]:
class Agent():
    def __init__(self, state_size, action_size, dqn_type='DQN', replay_memory_size=1e5, batch_size=32, gamma=0.99,
    	learning_rate=1e-3, target_tau=2e-3, update_rate=4, seed=0):

        self.dqn_type = dqn_type
        self.state_size = state_size
        self.action_size = action_size
        self.buffer_size = int(replay_memory_size)
        self.batch_size = batch_size
        self.gamma = gamma
        self.learn_rate = learning_rate
        self.tau = target_tau
        self.update_rate = update_rate
        self.seed = random.seed(seed)

        self.network = DQN(state_size, action_size, seed).to(device)
        self.target_network = DQN(state_size, action_size, seed).to(device)
        self.optimizer = optim.Adam(self.network.parameters(), lr=self.learn_rate)

        # Replay memory
        self.memory = ReplayBuffer(action_size, self.buffer_size, self.batch_size, seed)

        # Initialize time step (for updating every UPDATE_EVERY steps)
        self.t_step = 0

    def step(self, state, action, reward, next_state, done):
        # Save experience in replay memory
        self.memory.add(state, action, reward, next_state, done)
        
        # Learn every UPDATE_EVERY time steps.
        self.t_step = (self.t_step + 1) % self.update_rate
        if self.t_step == 0:
            # If enough samples are available in memory, get random subset and learn
            if len(self.memory) > self.batch_size:
                experiences = self.memory.sample()
                self.learn(experiences, self.gamma)

    def act(self, state, eps=0.0):

        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.network.eval()
        with torch.no_grad():
            action_values = self.network(state)
        self.network.train()

        # Epsilon-greedy action selection
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))
        
	########################################################
    # LEARN() method
    # Update value parameters using given batch of experience tuples.
    def learn(self, experiences, gamma, DQN=True):
        states, actions, rewards, next_states, dones = experiences

        # Get Q values from current observations (s, a) using model nextwork
        Qsa = self.network(states).gather(1, actions)

        if (self.dqn_type == 'DDQN'):
        #Double DQN
            Qsa_prime_actions = self.network(next_states).detach().max(1)[1].unsqueeze(1)
            Qsa_prime_targets = self.target_network(next_states)[Qsa_prime_actions].unsqueeze(1)

        else:
        #Regular (Vanilla) DQN
            Qsa_prime_target_values = self.target_network(next_states).detach()
            Qsa_prime_targets = Qsa_prime_target_values.max(1)[0].unsqueeze(1)        

        
        # Compute Q targets for current states 
        Qsa_targets = rewards + (gamma * Qsa_prime_targets * (1 - dones))
        
        # Compute loss (error)
        loss = F.mse_loss(Qsa, Qsa_targets)

        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # ------------------- update target network ------------------- #
        self.soft_update(self.network, self.target_network, self.tau)


    ########################################################
    def soft_update(self, local_model, target_model, tau):

        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

## 2. Traininig the agent using Deep Q-Learning

In [65]:
import time

num_episodes=10000
epsilon=1.0
epsilon_min=0.05
epsilon_decay=0.99
scores = []
scores_average_window = 10      
solved_score = 14     

number_of_units = 5
n_actions = 5          

action_size = 5**5

env = Environment()
# env = Environment.from_simulation_session(agent)

state_size = env.reset().shape[0]

agent = Agent(state_size=state_size, action_size=action_size, dqn_type='DQN', batch_size=32, gamma=0.99,)


# loop from num_episodes
for i_episode in range(1, num_episodes+1):
    
    reward_episode = []
    state = env.reset()     
    # state = env.get_state()
    # print(f"state: {state}")
    # print(f"env_info: {env_info}")
    
    # set the initial episode score to zero.
    score = 0

    while True:
        # determine epsilon-greedy action from current sate
        action = agent.act(state, epsilon)         
        # print(f"\naction: {action}")    

        # send the action to the environment and receive resultant environment information
        state, reward, done, ending = env.step(
            action=int(action),
            number_of_units=number_of_units,
            number_of_actions_per_unit=n_actions,)    

        # next_state = env_info.vector_observations[0]   # get the next state
        next_state = state
        
        # agent.learn(state, action, reward, next_state, done)
        # reward = env_info.rewards[0]   
        reward_episode.append(reward)
        # done = env_info.local_done[0]                  # see if episode has finished

        #Send (S, A, R, S') info to the DQN agent for a neural network update
        agent.step(state, action, reward, next_state, done)

        # set new state to current state for determining next action
        state = next_state

        # Update episode score
        score += reward

        # If unity indicates that episode is done, 
        # then exit episode loop, to begin new episode
        if done:
            break

    # Calculate mean score over last 100 episodes 
    # Mean score is calculated over current episodes until i_episode > 100
    scores.append(score)
    average_score = np.mean(scores[i_episode-min(i_episode,scores_average_window):i_episode+1])

    # Decrease epsilon for epsilon-greedy policy by decay rate
    # Use max method to make sure epsilon doesn't decrease below epsilon_min
    epsilon = max(epsilon_min, epsilon_decay*epsilon)

    # (Over-) Print current average score
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, average_score), end="")

    # Print average score every scores_average_window episodes
    if i_episode % scores_average_window == 0:
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, average_score))
    
    # Check to see if the task is solved (i.e,. avearge_score > solved_score). 
    # If yes, save the network weights and scores and end training.
    if average_score >= solved_score:
        print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, average_score))

        # Save trained neural network weights
        timestr = time.strftime("%Y%m%d-%H%M%S")
        nn_filename = "dqnAgent_Trained_Model_" + timestr + ".pth"
        torch.save(agent.network.state_dict(), nn_filename)

        # Save the recorded Scores data
        scores_filename = "dqnAgent_scores_" + timestr + ".csv"
        np.savetxt(scores_filename, scores, delimiter=",")
        break

Episode 5	Average Score: -680000.00

/Users/maksymleshchenko/Data Science/University/War games/war_games/apps/actor_critic_simulation/real_environment.py:26: RuntimeWarning: invalid value encountered in scalar divide
  avg_dist = np.sum(pairwise_distances) / (num_points * (num_points - 1))


Episode 10	Average Score: -712550.00
Episode 20	Average Score: -743150.00
Episode 30	Average Score: -644350.00
Episode 40	Average Score: -674100.00
Episode 50	Average Score: -673350.00
Episode 60	Average Score: -594300.00
Episode 70	Average Score: -695100.00
Episode 80	Average Score: -637900.00
Episode 90	Average Score: -710500.00
Episode 100	Average Score: -613350.00
Episode 110	Average Score: -550700.00
Episode 120	Average Score: -600300.00
Episode 130	Average Score: -508300.00
Episode 140	Average Score: -593800.00
Episode 150	Average Score: -670300.00
Episode 160	Average Score: -549200.00
Episode 170	Average Score: -484800.00
Episode 180	Average Score: -478450.00
Episode 190	Average Score: -533150.00
Episode 200	Average Score: -485100.00
Episode 210	Average Score: -470000.00
Episode 220	Average Score: -537050.00
Episode 230	Average Score: -553450.00
Episode 240	Average Score: -463100.00
Episode 250	Average Score: -488150.00
Episode 260	Average Score: -498500.00
Episode 270	Average S

KeyboardInterrupt: 

Agent didn't converge